# 6 WorkFlow

### 6.1 Objetivo

Presentar un workflow/pipeline completo al que los estudiantes deberán enriquecer

#### 6.2  Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Type -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [ ]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Mounted at /content/.drive


Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab
*   Bajar el **dataset_historico** al Google Drive y tambien al disco local de la virtual machine que esta corriendo Google Colab



In [ ]:
%%shell

mkdir -p "/content/.drive/My Drive/labo3"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo3" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets


webfiles="https://storage.googleapis.com/open-courses/austral2025-af91/"
destino_local="/content/datasets"
destino_bucket="/content/buckets/b1/datasets"


archivo="dataset_pequeno.csv"

if ! test -f $destino_bucket/$archivo; then
  wget  $webfiles/$archivo  -O $destino_bucket/$archivo
fi


if ! test -f $destino_local/$pequeno; then
  cp  $destino_bucket/$archivo  $destino_local/$archivo
fi

#-------

archivo="gerencial_competencia_2025.csv.gz"

if ! test -f $destino_bucket/$archivo; then
  wget  $webfiles/$archivo  -O $destino_bucket/$archivo
fi


if ! test -f $destino_local/$pequeno; then
  cp  $destino_bucket/$archivo  $destino_local/$archivo
fi


## 6.3  Workflow

## Inicializacion

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

limpio el ambiente de R

In [ ]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Sat Nov 22 10:59:05 PM 2025"

In [ ]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,660358,35.3,1454494,77.7,1454494,77.7
Vcells,1226536,9.4,8388608,64.0,1975136,15.1


In [ ]:
require("data.table")

if( !require("R.utils")) install.packages("R.utils")
require("R.utils")

if( !require("dplyr")) install.packages("dplyr")
require("dplyr")


Loading required package: data.table

Loading required package: R.utils

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘R.utils’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘R.oo’, ‘R.methodsS3’


Loading required package: R.utils

Loading required package: R.oo

Loading required package: R.methodsS3

R.methodsS3 v1.8.2 (2022-06-13 22:00:14 UTC) successfully loaded. See ?R.methodsS3 for help.

R.oo v1.27.1 (2025-05-02 21:00:05 UTC) successfully loaded. See ?R.oo for help.


Attaching package: ‘R.oo’


The following object is masked from ‘package:R.methodsS3’:

    throw


The following objects are masked from ‘package:methods’:

    getClasses, getMethods


The following objects are masked from ‘package:base’:

    attach, detach, load, save


R.utils v2.13.0 (2025-02-24 21:20:02 UTC) successfully loaded. See ?R.utils for help.




#### Parametros
Si es gerente, no cambie nada
<br>Si es Analista, cambie el nombre del dataset

In [ ]:
PARAM <- list()
PARAM$semilla_primigenia <- c(100151, 100237, 100291, 345689, 655883, 334561, 478453, 627269, 778889 ,819509)

PARAM$experimento <- 0002
PARAM$dataset <- "gerencial_competencia_2025.csv.gz"

#### Carpeta del Experimento

In [ ]:
# carpeta de trabajo

setwd("/content/buckets/b1/exp")
experimento_folder <- paste0("LGBM", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

### 6.3.1   Preprocesamiento del dataset

#### 6.3.1.1  DT incorporar dataset

In [ ]:
# lectura del dataset
dataset <- fread(paste0("/content/datasets/", PARAM$dataset))
print(dataset)

        numero_de_cliente foto_mes internet cliente_edad cliente_antiguedad
                    <int>    <int>    <int>        <int>              <int>
     1:          29186441   202005        1           61                282
     2:          29187961   202005        1           61                312
     3:          29193101   202005        1           67                365
     4:          29193281   202005        1           54                158
     5:          29198891   202005        1           54                312
    ---                                                                    
273662:         187838971   202109        0           49                  1
273663:         187902361   202109        0           22                  1
273664:         187956461   202109        0           34                  1
273665:         187964701   202109        0           26                  1
273666:         188116871   202109        0           30                  1
        mren

### 6.3.2 Modelado, producción y scoring

In [ ]:
PARAM$trainingstrategy$validate <- c(202107)

PARAM$trainingstrategy$training <- c(
  202106, 202105, 202104, 202103, 202102, 202101,
  202012, 202011, 202010, 202009, 202008, 202007,
  202006, 202005
)

PARAM$trainingstrategy$training_pct <- 1.0


PARAM$trainingstrategy$positivos <- c( "BAJA+1", "BAJA+2")

In [ ]:
# seteo la clase01   1={BAJA+1, BAJA+2}   0={CONTINUA}
dataset[, clase01 := ifelse( clase_ternaria %in% PARAM$trainingstrategy$positivos, 1, 0 )]

In [ ]:
# los campos en los que se entrena
campos_buenos <- copy( setdiff(
    colnames(dataset), c("clase_ternaria","clase01","azar"))
)

In [ ]:
if( !require("lightgbm")) install.packages("lightgbm")
require("lightgbm")

Loading required package: lightgbm

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘lightgbm’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: lightgbm



In [ ]:
for(seed in PARAM$semilla_primigenia) {

  cat("Ejecutando con semilla:", seed, "\n")

  # fijar semilla
  set.seed(seed, kind = "L'Ecuyer-CMRG")

  # generar azar para undersampling
  dataset[, azar := runif(nrow(dataset))]

  # undersampling para TRAIN
  dataset[, fold_train :=
            foto_mes %in% PARAM$trainingstrategy$training &
            (clase_ternaria %in% c("BAJA+1", "BAJA+2") |
             azar < PARAM$trainingstrategy$training_pct)
  ]

  # ---- TRAIN ----
  dtrain <- lgb.Dataset(
    data  = data.matrix(dataset[fold_train == TRUE, campos_buenos, with = FALSE]),
    label = dataset[fold_train == TRUE, clase01],
    free_raw_data = TRUE
  )

  # ---- VALIDATION ----
  dvalidate <- lgb.Dataset(
    data  = data.matrix(dataset[foto_mes %in% PARAM$trainingstrategy$validate,
                                campos_buenos, with = FALSE]),
    label = dataset[foto_mes %in% PARAM$trainingstrategy$validate, clase01],
    free_raw_data = TRUE
  )

  # ---- PARÁMETROS LGBM ----
  PARAM$lgbm$param_fijos <- list(
    objective = "binary",
    metric = "auc",
    first_metric_only = TRUE,
    boost_from_average = TRUE,
    feature_pre_filter = FALSE,
    verbosity = -100,
    force_row_wise = TRUE,
    seed = seed,
    max_bin = 31,
    feature_fraction = 0.5,
    num_leaves = 256,
    min_data_in_leaf = 107,
    learning_rate = 0.0063,
    max_depth = 17,
    min_sum_hessian_in_leaf = 0.8994,
    scale_pos_weight = 1.1414
  )

  # ---- FINAL TRAIN (SIN undersampling) ----
  PARAM$trainingstrategy$final_train <- c(
    202107,
    202106, 202105, 202104, 202103, 202102, 202101,
    202012, 202011, 202010, 202009, 202008, 202007,
    202006, 202005
  )

  dataset[, fold_final_train := foto_mes %in% PARAM$trainingstrategy$final_train ]

  dfinal_train <- lgb.Dataset(
    data  = data.matrix(dataset[fold_final_train == TRUE, campos_buenos, with = FALSE]),
    label = dataset[fold_final_train == TRUE, clase01],
    free_raw_data = TRUE
  )

  cat("Tamaño final_train:", nrow(dataset[fold_final_train == TRUE]), "\n")

  # ---- ENTRENAMIENTO FINAL ----
  final_model <- lgb.train(
    data  = dfinal_train,
    param = PARAM$lgbm$param_fijos,
    verbose = -100
  )

  cat("Modelo final entrenado con semilla:", seed, "\n")

  # grabo a disco el modelo en un formato para seres humanos ... ponele ...
  archivo_modelo <- paste0("modelo_", seed, ".txt")
  lgb.save(final_model, archivo_modelo)

  # IMPORTANCIA
  tb_importancia <- as.data.table(lgb.importance(final_model))
  archivo_importancia <- paste0("impo_", seed, ".txt")
  fwrite(tb_importancia, file = archivo_importancia, sep = "\t")

  cat("Archivos guardados para semilla:", seed, "\n")

  PARAM$trainingstrategy$future <- c(202109)

  dfuture <- dataset[ foto_mes %in% PARAM$trainingstrategy$future ]

  # aplico final_model a dfuture
  prediccion <- predict(
    final_model,
    data.matrix(dfuture[, campos_buenos, with = FALSE])
  )

  tb_prediccion <- dfuture[, list(numero_de_cliente)]
  tb_prediccion[, prob := prediccion]

  # archivo de predicción por semilla
  archivo_pred <- paste0("prediccion_", seed, ".txt")

  fwrite(
    tb_prediccion,
    file = archivo_pred,
    sep = "\t"
  )

# ============================================================
#   GENERACIÓN ARCHIVOS KAGGLE (por semilla)
# ============================================================

PARAM$kaggle$competencia <- "labo-i-2025-virtual-gerencial"
PARAM$kaggle$cortes <- seq(800, 1300, by = 50)

# ordeno por probabilidad descendente
setorder(tb_prediccion, -prob)

dir.create("kaggle", showWarnings = FALSE)

for (envios in PARAM$kaggle$cortes) {

  tb_prediccion[, Predicted := 0L]
  tb_prediccion[1:envios, Predicted := 1L]

  archivo_kaggle <- paste0(
    "./kaggle/LGBM",
    PARAM$experimento, "_",
    seed, "_",
    envios, ".csv"
  )

  fwrite(
    tb_prediccion[, list(numero_de_cliente, Predicted)],
    file = archivo_kaggle,
    sep = ","
  )

  # comando de subida a Kaggle
  comando <- "kaggle competitions submit"
  competencia <- paste("-c", PARAM$kaggle$competencia)
  arch <- paste("-f", archivo_kaggle)

  mensaje <- paste0(
    "-m 'envios=", envios,
    "  semilla=", seed,
    "'"
  )

  linea <- paste(comando, competencia, arch, mensaje)

  salida <- system(linea, intern = TRUE)
  cat(salida, "\n")
}

}


Ejecutando con semilla: 819509 
Tamaño final_train: 240562 
Modelo final entrenado con semilla: 819509 
Archivos guardados para semilla: 819509 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
